In [49]:
import sys

sys.path.append("/Users/ansel/ai/podcast-pipeline")
from pathlib import Path

from chonkie import LateChunker

from app.utils import merge_srt

chunker = LateChunker(
    embedding_model="all-MiniLM-L6-v2",
    mode="sentence",
    chunk_size=512,
    min_sentences_per_chunk=1,
    min_characters_per_sentence=12,
    delim="\n",
)


def chunks_to_text(chunks):
    chunk_text = [chunk.text for chunk in chunks]
    return "\n".join(chunk_text)


def srt_to_text(file_path):
    transcript = merge_srt(file_path)
    content_with_speaker = [
        f"{speaker}: {speech}" for idx, start, end, speaker, speech in transcript
    ]
    return "\n".join(content_with_speaker)

In [ ]:
episodes = [
    "030",
    "051",
    "061",
    "069",
    "098",
    "118",
    "186",
    "240",
    "298",
    "398",
    "405",
    "430",
]

files_dir = Path("../files/bulk/rotl/")

files = [
    file
    for file in sorted(files_dir.iterdir())
    # if any(episode in str(file) for episode in episodes)
    if ".srt" in str(file)
]

out_dir = Path("../files/chunked")
out_dir.mkdir(exist_ok=True)

for file in files:
    file_name = file.stem + ".txt"
    out_path = out_dir / file_name
    transcript = srt_to_text(file)
    chunks = chunker(transcript)
    text = chunks_to_text(chunks)
    with open(out_path, "w") as f:
        f.write(text)

In [ ]:
from pathlib import Path

import chromadb

client = chromadb.PersistentClient(path="../chromadb")

collection = client.get_or_create_collection("podcasts")

files_dir = Path("../files/chunked/")

files = [file for file in sorted(files_dir.iterdir())]

for file in files:
    file_name = file.name
    episode = file_name.split("_-_")[0]
    chunks = open(file, "r").read().split("\n\n")
    metadatas = [{"episode": episode} for i, chunk in enumerate(chunks)]
    ids = [str(i) for i, chunk in enumerate(chunks)]
    collection.add(documents=chunks, metadatas=metadatas, ids=ids)